In [ ]:
#### 2.1. Sequential version

In [1]:
from numpy.lib.function_base import interp
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from IMPACT import utils as utils_IMPACT
utils_IMPACT.set_seed(0)
from IMPACT import dataset
from IMPACT import model
import optuna
import logging
import gc
import json
import torch
import pandas as pd
from importlib import reload
from micat.dataset.preprocessing_utilities import *
from micat import utils as utils_micat
from functools import partial

CUDA is not available. Skipping CUDA seed setting.
CUDA is not available. Skipping CUDA seed setting.


In [2]:
utils_IMPACT.setuplogger(verbose = True, log_name="micat_postcovid")

In [3]:
%%time
config = utils_IMPACT.generate_hs_config(dataset_name='postcovid', esc = 'error', valid_metric= 'rmse',pred_metrics = ['rmse'])

utils_IMPACT.set_seed(config["seed"])
logging.info(config['dataset_name'])
train_data,valid_data,concept_map,metadata = load_dataset(config)

study = optuna.create_study(
    directions=["minimize"],  # Warning : specify directions for each objective (depends on the validation metric)
)
gc.collect()
torch.cuda.empty_cache()
objective_with_args = partial(utils_micat.IMPACT_pre_train, config=config, train_data=train_data, valid_data=valid_data)
study.optimize(objective_with_args, n_trials=1, n_jobs=1, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
pareto_trials = study.best_trials

logging.info(f"Best trial for {config['dataset_name']} : {study.best_trials}")
for trial in pareto_trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  Metric value: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")

CUDA is not available. Using CPU.
CUDA is not available. Skipping CUDA seed setting.
[INFO 26:05] postcovid


[I 2025-05-07 11:26:06,380] A new study created in memory with name: no-name-891ec552-25d4-427c-8dcc-81892f58ac70


CUDA is not available. Skipping CUDA seed setting.
[INFO 26:06] train on cpu
[INFO 26:06] -- START Training --
[INFO 26:31] -- END Training --
[INFO 26:31] -------Trial number : 0
Best epoch : 8
Values : [tensor(0.2805)]
Params : {'learning_rate': 0.015115779567223512, 'lambda': 1.0482104968991887e-07}


[I 2025-05-07 11:26:31,865] Trial 0 finished with value: 0.2804945707321167 and parameters: {'learning_rate': 0.015115779567223512, 'lambda': 1.0482104968991887e-07}. Best is trial 0 with value: 0.2804945707321167.


[INFO 26:31] Best trial for postcovid : [FrozenTrial(number=0, state=1, values=[0.2804945707321167], datetime_start=datetime.datetime(2025, 5, 7, 11, 26, 6, 444065), datetime_complete=datetime.datetime(2025, 5, 7, 11, 26, 31, 865743), params={'learning_rate': 0.015115779567223512, 'lambda': 1.0482104968991887e-07}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.05, log=True, low=1e-05, step=None), 'lambda': FloatDistribution(high=1e-05, log=True, low=1e-07, step=None)}, trial_id=0, value=None)]
[INFO 26:31] Trial #0
[INFO 26:31]   Metric value: [0.2804945707321167]
[INFO 26:31]   Params: {'learning_rate': 0.015115779567223512, 'lambda': 1.0482104968991887e-07}
CPU times: user 3min 4s, sys: 20.5 s, total: 3min 24s
Wall time: 26.7 s


In [4]:
#### 2.2. Parallelized version

In [5]:
## Launch cluster with the following command at the root of the project :
#ipcluster start --n=3
#ipcluster stop

In [3]:
import importlib
import IMPACT.utils as utils_IMPACT
import IMPACT.model as model_IMPACT
import IMPACT.dataset as dataset_IMPACT

from micat import utils as utils_micat, dataset as dataset_micat
from functools import partial

from ipyparallel import Client
import dill
import json
import logging
import gc
import torch
from micat.dataset.preprocessing_utilities import *
cat_absolute_path = os.path.abspath('../../')

rc = Client()
rc[:].use_dill()
lview = rc.load_balanced_view()


rc[:].execute("import sys; sys.path.append('"+cat_absolute_path+"')")
logging.info("sys.path.append("+cat_absolute_path+")")
with rc[:].sync_imports():
    import json
    from IMPACT import utils as utils_IMPACT, model as model_IMPACT, dataset as dataset_IMPACT
    import logging
    import gc
    import torch

importing json on engine(s)
importing utils,model,dataset from IMPACT on engine(s)
importing logging on engine(s)
importing gc on engine(s)
importing torch on engine(s)


In [4]:
%%time
config = utils_IMPACT.generate_hs_config(dataset_name='postcovid', esc = 'error', valid_metric= 'rmse',pred_metrics = ['rmse'])

utils_IMPACT.set_seed(config["seed"])
logging.info(config['dataset_name'])
train_data,valid_data,concept_map,metadata = load_dataset(config)

study = optuna.create_study(
    directions=["minimize"], # Warning : specify directions for each objective (depends on the validation metric)
)
gc.collect()
torch.cuda.empty_cache()
objective_with_args = partial(utils_micat.objective_hps,config=config, train_data=train_data, valid_data=valid_data)
study.optimize(objective_with_args, n_trials=1, n_jobs=3, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
pareto_trials = study.best_trials

logging.info(f"Best trial for {config['dataset_name']} : {study.best_trials}")
for trial in pareto_trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  Metric value: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")

CUDA is not available. Using CPU.
CUDA is not available. Skipping CUDA seed setting.


[I 2025-05-07 11:35:45,172] A new study created in memory with name: no-name-08cf3c98-2d9a-4faf-9f62-ad956ae54c73
[I 2025-05-07 11:37:05,319] Trial 0 finished with value: 0.5218191146850586 and parameters: {'learning_rate': 0.0007822289807029084, 'lambda': 1.5704129766851874e-07}. Best is trial 0 with value: 0.5218191146850586.


CPU times: user 1.78 s, sys: 725 ms, total: 2.51 s
Wall time: 1min 21s
